In [16]:
import requests
from lxml import html
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import seaborn as sns

import plotly.graph_objs as go
from plotly.offline import iplot

In [17]:
# Create a session for persistent connection
session = requests.Session()
adapter = requests.adapters.HTTPAdapter(pool_connections=100, pool_maxsize=100)
session.mount('http://', adapter)
session.mount('https://', adapter)



In [18]:
# Functions

def import_xml(url, xpaths, session):
    data = []
    try:
        response = session.get(url, timeout=10)
        response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
        tree = html.fromstring(response.content)
        for xpath in xpaths:
            elements = tree.xpath(xpath)
            if elements:
                text = elements[0].text_content().strip("%)(").replace("(", "-")
                data.append(str(text))
            else:
                data.append("nan")
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        data = ["nan"] * len(xpaths)  # If the request fails, return 'nan' for all items
    return data

def get_quick_data(ticker, session):
    url = f"https://www.marketwatch.com/investing/stock/{ticker}/company-profile"
    xpaths = ["/html/body/div[3]/div[6]/div[2]/div[2]/div[1]/table/tbody/tr[7]/td[2]",#Enterprise Value to EBITDA
                "/html/body/div[3]/div[6]/div[2]/div[3]/div[1]/table/tbody/tr[8]/td[2]",#Return on Invested Capital
                #"/html/body/div[3]/div[6]/div[2]/div[3]/div[1]/table/tbody/tr[5]/td[2]",#Return on Assets
                #"/html/body/div[3]/div[6]/div[2]/div[3]/div[1]/table/tbody/tr[1]/td[2]",#Gross Margin
                "/html/body/div[3]/div[6]/div[2]/div[3]/div[1]/table/tbody/tr[4]/td[2]",#Net Margin
                "/html/body/div[3]/div[6]/div[2]/div[2]/div[1]/table/tbody/tr[5]/td[2]",#Price to Book Ratio
                #"/html/body/div[3]/div[6]/div[2]/div[3]/div[2]/table/tbody/tr[4]/td[2]",#Long-Term Debt to Equity
                "/html/body/div[3]/div[6]/div[2]/div[2]/div[3]/table/tbody/tr[2]/td[2]",#Quick Ratio
                #"/html/body/div[3]/div[6]/div[1]/div[1]/div/ul/li[6]/span",# 2022 sales growth
                #"/html/body/div[3]/div[6]/div[2]/div[2]/div[1]/table/tbody/tr[8]/td[2]",# EV to sales
                #"/html/body/div[3]/div[6]/div[2]/div[2]/div[2]/table/tbody/tr[2]/td[2]",# income per employee
                #"/html/body/div[3]/div[6]/div[1]/div[1]/div/ul/li[2]/span",#Sector
                "/html/body/div[3]/div[6]/div[1]/div[1]/div/ul/li[1]/span",#Industry
                #"/html/body/div[3]/div[6]/div[2]/div[4]/table/tbody/tr[2]/td[2]/div[1]/span[1]",#buys_6m
                #"/html/body/div[3]/div[6]/div[2]/div[4]/table/tbody/tr[1]/td[2]/div[1]/span[1]"]#buys_3m
    ]
    return import_xml(url, xpaths, session)



In [19]:
def batch_get_quick_data(tickers):
    data_list = []
    for ticker in tickers:
        data = [ticker, *get_quick_data(ticker, session)]
        data_list.append(data)
    return data_list

In [20]:
def uniform_magnitudes(value: str) -> float:
    """Converts a string with magnitude notation (B, M, T) to a float representation in billions."""
    magnitude_dict = {"B": 1, "K": 1/1000000, "M": 1/1000, "T": 1000}
    
    value = str(value).strip()
    
    if value.startswith("$"):
        value = value[1:]
        
    if value.startswith("€"):
        value = value[1:]
        
    if value.startswith("£"):
        value = value[1:]
    
    if value.startswith("(") and value.endswith(")"):
        value = "-" + value[1:-1]

    if value[-1] in magnitude_dict:
        value = float(value[:-1]) * magnitude_dict[value[-1]]
    else:
        #value = float(value)
        value=np.NaN
    
    return value

In [21]:
def gather_data(lists: list):
    i = 1
    for lst in lists:
        # Retrieve data
        data = batch_get_quick_data(lst)

        # Convert to DataFrame for better visualization and manipulation
        df = pd.DataFrame(data, columns=['Ticker', 'EV/EBITDA', 'ROIC', 'Net Margin', 'P/B Ratio',
                                         'Quick Ratio',"Industry"])
        
        # Ensure numeric columns are of type float
        numeric_cols = ['EV/EBITDA', 'ROIC', 'Net Margin', 'P/B Ratio',
                                         'Quick Ratio']
        for col in numeric_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
        df = df.dropna(axis=0)
        columns_to_consider = ['EV/EBITDA', 'ROIC', 'Net Margin', 'P/B Ratio',
                                         'Quick Ratio']

# Dropping rows where columns 'A' and 'C' have NaN values, ignoring NaNs in 'B'
        df = df.dropna(subset=columns_to_consider,axis=0)
        df = df.sort_values(by='ROIC', ascending=False)

    return df

## Insider Buys Selection

The tickers selected below are companies for which insiders have been recently buying.

The information in insider buys is obtained through www.dataroma.com


In [22]:
insider_bought  = [
    "SGHT", "CRGY", "HHH", "ITW", "DKS", "BMY", "TMTC", "TTSH", "HTLD", "CTRN",
    "ABR", "CTVA", "EXEL", "FMC", "UEIC", "RWAY", "SPB", "QRVO", "KMPR", "PRGO",
    "TLYS", "XRAY", "MODG", "COMM", "TCBI", "YUMC", "POST", "CRL", "AMRC", "XERS",
    "DAR", "DOW", "TRU", "SMRT", "TPL", "DBD", "BRCC", "GCI", "EMR", "UHAL",
    "BSM", "RILY", "BILL", "ZLAB", "CHRW", "HAIN", "MTDR", "T", "CTOS", "AAN",
    "UTZ", "VVV", "ADV", "ET", "MTD", "BYON", "TKO", "ATRA", "ED", "QRHC",
    "NTRS", "PACK", "USB", "B", "SPG", "NVST", "SNCR", "SWX", "SPWH", "MKL",
    "D", "OSTK", "AES", "SKIL", "RGF","BALL","WOOF","SCVL","CTRN","TLYS","HI","ECL","DINO","HTLD",
    "AAP","UEIC"
]

insider_bought=list(set(insider_bought))


In [23]:
df_insider=gather_data([insider_bought])
print(df_insider.shape)
df_insider.head()

(66, 7)


,Ticker,EV/EBITDA,ROIC,Net Margin,P/B Ratio,Quick Ratio,Industry
76,TPL,18.87,62.52,66.88,23.34,16.00,Upstream Oil & Gas
68,BSM,5.16,40.17,60.75,3.88,5.64,Upstream Oil & Gas
43,MTDR,3.10,31.37,37.98,2.19,1.83,Upstream Oil & Gas
14,ITW,19.25,30.25,19.04,21.76,0.93,Diversified Holding Companies
59,CHRW,8.91,30.12,3.81,7.87,1.08,Transportation Services


In [24]:
def add_column_from_web(df, url_prefix, url_suffix, xpath, new_column_name):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    def fetch_data(ticker):
        try:
            url = f"{url_prefix}{ticker}{url_suffix}"
            page = requests.get(url, headers=headers)
            if page.status_code == 200:
                tree = html.fromstring(page.content)
                data = tree.xpath(xpath)
                return data[0].text_content().strip() if data else 0
            else:
                print(f"Failed to fetch {url}: Status code {page.status_code}")
                return 0
        except Exception as e:
            print(f"Error processing {ticker}: {e}")
            return 0

    df[new_column_name] = df['Ticker'].apply(fetch_data)
    return df

# Example usage
# df = pd.DataFrame({'Ticker': ['AAPL', 'MSFT', 'GOOGL']})
# url_prefix = "https://www.example.com/stock?sym="
# url_suffix = "&info=data"
# xpath = "/html/body/div[1]/div[3]/div[3]/table/tbody/tr/td[3]"
# new_column_name = "NewData"
# new_df = add_column_from_web(df, url_prefix, url_suffix, xpath, new_colum



In [25]:
def apply_function_to_column(df, column_name, custom_function):
    # Applying the custom function to the specified column
    df[column_name] = df[column_name].apply(custom_function)
    return df


In [26]:
df_insider=add_column_from_web(df_insider,"https://www.marketwatch.com/investing/stock/","","/html/body/div[3]/div[6]/div[1]/div[1]/div/ul/li[4]/span[1]","Market Cap")
df_insider=add_column_from_web(df_insider,"https://www.marketwatch.com/investing/stock/","","/html/body/div[3]/div[6]/div[1]/div[1]/div/ul/li[11]/span[1]","Dividend Yield")
df_insider=add_column_from_web(df_insider,"https://www.marketwatch.com/investing/stock/","/financials/cash-flow","/html/body/div[3]/div[6]/div/div[4]/div/div/table/tbody/tr[23]/td[4]/div/span","FCF_2021")
df_insider=add_column_from_web(df_insider,"https://www.marketwatch.com/investing/stock/","/financials/cash-flow","/html/body/div[3]/div[6]/div/div[4]/div/div/table/tbody/tr[23]/td[5]/div/span","FCF_2022")
df_insider=add_column_from_web(df_insider,"https://www.marketwatch.com/investing/stock/","/financials/cash-flow","/html/body/div[3]/div[6]/div/div[4]/div/div/table/tbody/tr[23]/td[6]/div/span","FCF_2023")


df_insider.head()


,Ticker,EV/EBITDA,ROIC,Net Margin,P/B Ratio,Quick Ratio,Industry,Market Cap,Dividend Yield,FCF_2021,FCF_2022,FCF_2023
76,TPL,18.87,62.52,66.88,23.34,16.00,Upstream Oil & Gas,$11.4B,0.86%,201.95M,249.62M,427.94M
68,BSM,5.16,40.17,60.75,3.88,5.64,Upstream Oil & Gas,$3.36B,11.92%,276.99M,242.25M,412.42M
43,MTDR,3.10,31.37,37.98,2.19,1.83,Upstream Oil & Gas,$6.37B,1.50%,(302.87M),319.88M,970.57M
14,ITW,19.25,30.25,19.04,21.76,0.93,Diversified Holding Companies,$74.29B,2.26%,2.57B,2.26B,1.94B
59,CHRW,8.91,30.12,3.81,7.87,1.08,Transportation Services,$9.97B,2.84%,476.06M,60.76M,1.59B


In [27]:
def calculate_fcf_cv(df, col1, col2, col3):
    # Calculate mean and standard deviation across the specified columns
    df['Mean_FCF'] = df[[col1, col2, col3]].mean(axis=1)
    df['Std_FCF'] = df[[col1, col2, col3]].std(axis=1)

    # Calculate Coefficient of Variation
    df['FCF_CV'] = (df['Std_FCF'] / df['Mean_FCF']) * 100
    
    return None

In [28]:
df_insider=apply_function_to_column(df_insider, "Market Cap", uniform_magnitudes)
df_insider=apply_function_to_column(df_insider, "FCF_2021", uniform_magnitudes)
df_insider=apply_function_to_column(df_insider, "FCF_2022", uniform_magnitudes)
df_insider=apply_function_to_column(df_insider, "FCF_2023", uniform_magnitudes)

df_insider=df_insider[["Ticker","EV/EBITDA","ROIC","Net Margin","Market Cap","Dividend Yield","Industry","P/B Ratio","FCF_2021","FCF_2022","FCF_2023"]]

df_insider = df_insider.sort_values(by='Market Cap')

df_insider=df_insider.reset_index(drop=True)

df_insider.head(df_insider.shape[0])

,Ticker,EV/EBITDA,ROIC,Net Margin,Market Cap,Dividend Yield,Industry,P/B Ratio,FCF_2021,FCF_2022,FCF_2023
0,SNCR,5.57,-2.50,-3.13,0.04797,N/A,Software,0.82,-0.00145,0.00342,0.01595
1,RGF,-3.50,-10.24,-7.76,0.05701,N/A,Food Products,1.21,-0.02929,-0.06101,NaN
2,ATRA,0.41,-93.21,-359.12,0.06726,N/A,Biotechnology,2.48,-0.18396,-0.22924,-0.26661
3,UEIC,3.42,0.14,0.07,0.09844,N/A,Audio/Video Equipment,0.99,0.05653,0.02770,-0.00402
4,SKIL,10.07,-59.38,-142.32,0.12967,N/A,N/A,0.59,0.05511,-0.02585,NaN
...,...,...,...,...,...,...,...,...,...,...,...
61,EMR,13.54,9.00,14.15,50.00000,2.37%,Precision Products,2.67,2.99000,2.62000,0.27400
62,ECL,24.60,6.93,7.69,54.26000,1.11%,Environment/Waste Management,5.72,1.37000,1.42000,1.08000
63,ITW,19.25,30.25,19.04,74.29000,2.26%,Diversified Holding Companies,21.76,2.57000,2.26000,1.94000
64,BMY,6.83,8.80,13.71,101.49000,4.49%,Pharmaceuticals,4.86,13.30000,15.23000,11.95000


In [29]:
df_insider=add_column_from_web( df_insider,"https://finance.yahoo.com/quote/","/key-statistics/",
                              "/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/div/div/section/div[2]/div[1]/div/div/div/div/table/tbody/tr[2]/td[2]",
                              "EV")
df_insider=apply_function_to_column( df_insider, "EV", uniform_magnitudes)

df_insider['FCFy_3yAvg'] = (( df_insider['FCF_2021'] +  df_insider['FCF_2022'] +  df_insider['FCF_2023']) / 3) /  df_insider['EV'] * 100
df_insider["FCFy_2023"] =  ( df_insider['FCF_2023'] /  df_insider['EV']) * 100

# Step 3: Reorder columns to make the new column the second column
col_order =  df_insider.columns.tolist()
col_order.insert(1, col_order.pop(col_order.index('FCFy_3yAvg')))
col_order.insert(2, col_order.pop(col_order.index("FCFy_2023")))
df_insider =  df_insider[col_order]
df_insider=add_column_from_web( df_insider,"https://finance.yahoo.com/quote/","/profile/",
                              "/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/div/div/section/div[1]/div/div/p[2]/span[4]",
                              "Industry")
# Step 4: Sort the DataFrame based on the new column in descending order
df_insider =  df_insider.sort_values(by='FCFy_3yAvg', ascending=False)
df_insider= df_insider.reset_index(drop=True)
df_insider= df_insider.round(2)
# Now df has the new column and is sorted accordingly
df_insider.head( df_insider.shape[0])

,Ticker,FCFy_3yAvg,FCFy_2023,EV/EBITDA,ROIC,Net Margin,Market Cap,Dividend Yield,Industry,P/B Ratio,FCF_2021,FCF_2022,FCF_2023,EV
0,UEIC,20.54,-3.09,3.42,0.14,0.07,0.10,N/A,Consumer Electronics,0.99,0.06,0.03,-0.00,0.13
1,DOW,11.00,11.56,5.87,13.02,8.01,35.69,5.48%,Chemicals,1.71,4.97,5.51,5.65,48.88
2,BYON,10.70,-2.95,14.36,-4.78,-1.83,1.03,N/A,Internet Retail,1.35,0.19,0.07,-0.02,0.74
3,CTRN,10.44,-4.24,4.45,17.69,7.41,0.21,0.00%,Apparel Retail,1.61,0.09,0.04,-0.02,0.39
4,BMY,10.14,8.98,6.83,8.80,13.71,101.49,4.49%,Drug Manufacturers - General,4.86,13.30,15.23,11.95,133.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,SKIL,NaN,NaN,10.07,-59.38,-142.32,0.13,N/A,Education & Training Services,0.59,0.06,-0.03,NaN,0.73
62,SMRT,NaN,NaN,-4.71,-23.68,-57.40,0.66,N/A,Software - Application,1.32,-0.08,NaN,NaN,0.73
63,BRCC,NaN,NaN,-4.66,-137.91,-27.51,0.95,N/A,Packaged Foods,13.66,-0.15,NaN,NaN,1.03
64,ADV,NaN,NaN,7.07,-35.60,-34.09,0.98,N/A,Advertising Agencies,0.59,0.09,0.08,NaN,2.73


## Ranking 

The stocks are ranked by FCFy considering the average FCF from 2021, 2022 and 2023

The FCF is divided by EV to obtain the yield, this accounts for the net debt as comparison factor

Fundamental data is taken from www.marketwatch.com and https://finance.yahoo.com/


In [30]:
df_insider.head(20)

,Ticker,FCFy_3yAvg,FCFy_2023,EV/EBITDA,ROIC,Net Margin,Market Cap,Dividend Yield,Industry,P/B Ratio,FCF_2021,FCF_2022,FCF_2023,EV
0,UEIC,20.54,-3.09,3.42,0.14,0.07,0.10,N/A,Consumer Electronics,0.99,0.06,0.03,-0.00,0.13
1,DOW,11.00,11.56,5.87,13.02,8.01,35.69,5.48%,Chemicals,1.71,4.97,5.51,5.65,48.88
2,BYON,10.70,-2.95,14.36,-4.78,-1.83,1.03,N/A,Internet Retail,1.35,0.19,0.07,-0.02,0.74
3,CTRN,10.44,-4.24,4.45,17.69,7.41,0.21,0.00%,Apparel Retail,1.61,0.09,0.04,-0.02,0.39
4,BMY,10.14,8.98,6.83,8.80,13.71,101.49,4.49%,Drug Manufacturers - General,4.86,13.30,15.23,11.95,133.08
5,DINO,9.25,30.30,2.91,26.76,7.57,11.57,3.41%,Oil & Gas Refining & Marketing,1.11,0.13,-0.40,3.26,10.76
6,BSM,9.24,12.27,5.16,40.17,60.75,3.36,11.92%,Oil & Gas E&P,3.88,0.28,0.24,0.41,3.36
7,AAP,8.73,4.28,6.82,7.94,4.50,3.28,1.78%,Specialty Retail,3.25,0.70,0.82,0.30,6.96
8,T,8.20,4.60,6.66,-2.85,-6.91,121.12,6.51%,Telecom Services,1.35,27.46,26.41,12.40,269.31
9,QRVO,8.09,6.31,16.11,1.63,2.89,9.49,N/A,Semiconductors,2.57,1.11,0.84,0.68,10.84
